# Importazione librerie

In [3]:
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
print("Importazione librerie completata")

Importazione librerie completata


# Pipeline recensione

In [4]:
class PipelineRecensioneFilm:
    def __init__(self, lingua='italian'):
        """Pipeline NLP per testi di recensioni cinematografiche"""
        self.lingua = lingua
        self.nlp = spacy.load('it_core_news_sm' if lingua == 'italian' else 'en_core_web_sm')
        self.stemmer = SnowballStemmer(lingua)
        self.stop_words = set(stopwords.words(lingua))
        
    def normalizza_testo(self, testo):
        """Normalizzazione base: minuscole e rimozione punteggiatura"""
        testo = testo.lower()
        testo = re.sub(r'[^\w\sàèéìòù]', ' ', testo)  # Rimuovi punteggiatura
        testo = re.sub(r'\s+', ' ', testo).strip()
        return testo
    
    def tokenizza_e_lemmatizza(self, testo):
        """Tokenizza e lemmatizza mantenendo solo parole significative"""
        doc = self.nlp(testo)
        tokens = []
        for token in doc:
            # escludi punteggiatura, spazi, parole brevi o non alfabetiche
            if not token.is_punct and not token.is_space and token.is_alpha:
                # prendi lemma (non -PRON-)
                lemma = token.lemma_ if token.lemma_ != '-PRON-' else token.text
                tokens.append(lemma)
        return tokens
    
    def rimuovi_stop_words(self, tokens):
        """Rimuove stop words mantenendo parole chiave semantiche"""
        return [t for t in tokens if t not in self.stop_words]
    
    def filtra_per_pos(self, testo):
        """Estrae solo nomi e aggettivi per analisi semantica"""
        doc = self.nlp(testo)
        return [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ']]
    
    def processa_testo(self, testo, filtra_pos=True):
        """Pipeline completa"""
        testo_norm = self.normalizza_testo(testo)
        tokens = self.tokenizza_e_lemmatizza(testo_norm)
        tokens = self.rimuovi_stop_words(tokens)
        if filtra_pos:
            # Mantiene solo sostantivi e aggettivi (per analisi semantica/sentiment)
            tokens = [t for t in tokens if t in self.filtra_per_pos(testo_norm)]
        return tokens


# Test

In [5]:
# Test 2 — Recensione di Film
testo2 = "La fotografia è meravigliosa e la colonna sonora accompagna perfettamente la storia, ma il ritmo narrativo risulta lento e prevedibile."

pipeline_film = PipelineRecensioneFilm()
risultato2 = pipeline_film.processa_testo(testo2)

print("🎬 Tokens finali:", risultato2)

🎬 Tokens finali: ['fotografia', 'meraviglioso', 'colonna', 'sonoro', 'storia', 'ritmo', 'narrativo', 'lento', 'prevedibile']


# Approccio utilizzato
- Niente dizionario di slang o emoji.
- Focus su lemmi, aggettivi e nomi (utile per analisi semantica/sentiment).
- Pulisci il testo in modo semplice (solo minuscole e punteggiatura).